In [1]:
import quippy
import ase
from ase.calculators.calculator import all_changes

import logging
logging.basicConfig(level=logging.WARNING)

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import os
import sys

In [4]:
sys.path.insert(0,"/home/es732/programs/function_modules/")
import mbx_functions

In [5]:
dirpath = os.getcwd()
print dirpath

/home/es732/programs/MBX/quip_interface/test


In [6]:
from collections import OrderedDict

In [7]:
d_list=[0.1,0.01,0.001,0.0001,0.00001,0.000001]

In [8]:
version_list=["mbx_pbc","mbx_pbc_no_mww_pip","mbx_pbc_no_www_mww","mbx_pbc_no_ww_mw_www_mww",
              "mbx_pbc_no_ww_mw","mbx_pbc_no_ww","mbx_pbc_no_mw","mbx_pbc_no_ww_www_mww"]

In [9]:
structure_list=["corner","central"]

In [10]:
dict_version_d_virial=OrderedDict([(struct,
                                    OrderedDict([(version,
                                        OrderedDict([(d, [[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                                 for d in d_list]))
                                       for version in version_list]))
                                    for struct in structure_list])

In [11]:
dict_version_model_virial=OrderedDict([(struct,
                                        OrderedDict([(version,[[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                       for version in version_list]))
                                       for struct in structure_list]) 

In [12]:
def calculate_numerical_virial_diag(pot, in_atoms, d=1e-2):
    ## modified from the function calculate_numerical_stress() at ase/calculators/calculator.py 
    """Calculate numerical stress using finite difference."""
    pot.reset()

    atoms=in_atoms.copy()
    stress = np.zeros((3, 3), dtype=float)

    cell = atoms.cell.copy()
    V = atoms.get_volume()
    for i in range(3):
        x = np.eye(3)
        x[i, i] += d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        atoms.set_calculator(pot)
        eplus = atoms.get_potential_energy()#force_consistent=True)
        pot.reset()

        x[i, i] -= 2 * d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        atoms.set_calculator(pot)
        eminus = atoms.get_potential_energy()#force_consistent=True)
        pot.reset()

        stress[i, i] = (eplus - eminus) / (2 * d * V)
        x[i, i] += d
    atoms.set_cell(cell, scale_atoms=True)
    virial= (-stress*atoms.get_volume())
    return virial

In [13]:
this_root="/home/es732/programs/MBX/quip_interface/test/"

In [14]:
this_struct="phase_clathrate_2x2x2_1_CH4.xyz"
this_at_tmp=quippy.Atoms(this_root+this_struct)
dict_pot=mbx_functions.create_mbx_potential_variables(this_at_tmp)

## test that energy is consistent

In [15]:
this_at_tmp.numbers[138]

6

In [16]:
pos_ch4_orig=this_at_tmp.positions[138]
print pos_ch4_orig
print type(pos_ch4_orig)

[5.7790244  0.15542982 2.95623668]
<type 'numpy.ndarray'>


In [21]:
import unwrap_oh2_ch4

In [32]:
struct_corner_tmp=this_at_tmp.copy()
for i in range(len(struct_corner_tmp)):
    struct_corner_tmp.positions[i]=this_at_tmp.positions[i]-pos_ch4_orig

struct_corner_tmp.wrap()

struct_corner_unwrapped = unwrap_oh2_ch4.unwrap_water_methane_diff_order(ase.Atoms(struct_corner_tmp.copy()))
struct_corner = ase.Atoms(struct_corner_unwrapped["phase_mw"] )
struct_corner=quippy.Atoms(struct_corner)

In [33]:
struct_corner.write("phase_clathrate_2x2x2_1_CH4_corner.xyz")

In [34]:
np.diag(this_at_tmp.cell)

array([23.4858, 23.4858, 23.4858])

In [35]:
pos_shift_middle=np.diag(this_at_tmp.cell)*0.5
pos_shift_middle-pos_ch4_orig

array([ 5.9638756 , 11.58747018,  8.78666332])

In [36]:
struct_central_tmp=this_at_tmp.copy()
for i in range(len(struct_central_tmp)):
    struct_central_tmp.positions[i]=this_at_tmp.positions[i]+(pos_shift_middle-pos_ch4_orig)
struct_central_tmp.wrap()
struct_central_unwrapped = unwrap_oh2_ch4.unwrap_water_methane_diff_order(ase.Atoms(struct_central_tmp.copy()))
struct_central = ase.Atoms(struct_central_unwrapped["phase_mw"] )
struct_central=quippy.Atoms(struct_central)

In [37]:
struct_central.write("phase_clathrate_2x2x2_1_CH4_central.xyz")

In [38]:
this_version=version_list[0]
print this_version
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

mbx_pbc


In [39]:
pot_MBX.reset()
pot_MBX.calc(struct_corner,energy=True,force=True,virial=True)
energy_corner=struct_corner.params["energy"]
forces_corner=struct_corner.properties["force"]
virial_corner= np.array( struct_corner.params["virial"])

In [40]:
pot_MBX.reset()
pot_MBX.calc(struct_central,energy=True,force=True,virial=True)
energy_central=struct_central.params["energy"]
forces_central=struct_central.properties["force"]
virial_central= np.array( struct_central.params["virial"])

In [41]:
energy_corner-energy_central

-3.4219738154206425e-11

In [42]:
np.max(forces_corner-forces_central)

FortranArray(5.14441267e-12)

In [43]:
virial_corner-virial_central

array([[-1.71951342e-11, -7.25038363e-11,  2.42047132e-11],
       [-7.25038363e-11, -2.66027200e-11,  6.20382079e-11],
       [ 2.42047132e-11,  6.20382079e-11, -2.09183781e-11]])

## diagonal_virial=F, only 2B mw PIP

In [44]:
this_version=version_list[7]
print this_version

mbx_pbc_no_ww_www_mww


In [45]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [46]:
this_struct="corner"
at= struct_corner.copy()

In [47]:
pot_MBX.reset()
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_struct][this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_struct][this_version]

[[-1130.27919495     2.23644373     4.32139573]
 [    2.23644373 -1147.06287748     5.36659804]
 [    4.32139573     5.36659804 -1142.27584258]]


In [48]:
for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_struct][this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [49]:
this_struct="central"
at= struct_central.copy()

In [50]:
pot_MBX.reset()
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_struct][this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_struct][this_version]

[[-1130.27919495     2.23644373     4.32139573]
 [    2.23644373 -1147.06287748     5.36659804]
 [    4.32139573     5.36659804 -1142.27584258]]


In [51]:
for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_struct][this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## Errors in derivative virial / Virial from model (only diagonal)

### model without PIPs - except mw

In [52]:
this_version="mbx_pbc_no_ww_www_mww"
this_struct="corner"
for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_struct][this_version][tmp_d] -dict_version_model_virial[this_struct][this_version])/dict_version_model_virial[this_struct][this_version])

d = 0.01
[-0.00015633 -0.00012878 -0.00017994]
d = 0.001
[9.11363718e-05 9.86282464e-05 4.55514104e-05]
d = 0.0001
[9.36141549e-05 1.00897074e-04 4.78069017e-05]
d = 1e-05
[9.36388565e-05 1.00920072e-04 4.78295843e-05]
d = 1e-06
[9.36360704e-05 1.00921058e-04 4.78275291e-05]


In [53]:
this_version="mbx_pbc_no_ww_www_mww"
this_struct="central"
for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_struct][this_version][tmp_d] -dict_version_model_virial[this_struct][this_version])/dict_version_model_virial[this_struct][this_version])

d = 0.01
[-0.00015633 -0.00012878 -0.00017994]
d = 0.001
[9.11363718e-05 9.86282464e-05 4.55514104e-05]
d = 0.0001
[9.36141549e-05 1.00897074e-04 4.78069017e-05]
d = 1e-05
[9.36388565e-05 1.00920072e-04 4.78295843e-05]
d = 1e-06
[9.36360704e-05 1.00921058e-04 4.78275291e-05]


In [55]:
this_version="mbx_pbc_no_ww_www_mww"
this_struct1="central"
this_struct2="corner"

for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_struct1][this_version][tmp_d] -dict_version_d_virial[this_struct2][this_version][tmp_d] ))

d = 0.01
[0. 0. 0.]
d = 0.001
[0. 0. 0.]
d = 0.0001
[0. 0. 0.]
d = 1e-05
[0. 0. 0.]
d = 1e-06
[0. 0. 0.]
